In [3]:
import os
import datetime
from miditime.miditime import MIDITime
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('lunar_Crater_Ages.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'lunar_Crater_Ages.csv'

In [35]:
df.describe()

,longitude,latitude,diameter,age
count,111.000000,111.000000,111.000000,111.000000
mean,167.443686,4.417544,18.957658,349.864865
std,100.608193,36.927960,15.602987,276.289390
min,4.526810,-73.446400,10.000000,4.000000
25%,80.550250,-19.862575,11.400000,138.500000
50%,162.419300,8.117410,13.200000,290.000000
75%,241.669110,27.267050,18.050000,503.000000
max,359.650470,75.429880,96.700000,1324.000000


In [36]:
df = df[['age', 'diameter']]
df.sort_values(by='age')

# Lets save cleaned and sorted dataframe to clean_lunar_Crater_Ages.csv
df.to_csv('clean_lunar_Crater_Ages.csv', index=False)

In [38]:
# Instantiate the MIDITime class
mymidi = MIDITime(120, 'data_sonified.mid', 7, 3, 4)

# Fill missing diameter values with 0
df['diameter'] = df['diameter'].fillna(0)

# Sort the DataFrame by the 'age' column
df = df.sort_values(by='age', ascending=True)

# Convert the DataFrame to a list of dictionaries
my_data = df.to_dict('records')

# Map 'age' to beats using MIDITime, and keep the 'diameter' value
my_data_timed = [
    {
        'beat': mymidi.beat(d['age']),
        'diameter': d['diameter']
    } for d in my_data
]

# Reference starting time (the first event's beat)
start_time = my_data_timed[0]['beat']

# Function to convert diameter to pitch (larger diameter = lower pitch)
def data_to_pitch_tuned(diameter):
    # Scale the diameter to a percentage where larger diameters map to lower notes
    scale_pct = mymidi.linear_scale_pct(96.7, 10, diameter)  # Higher diameter = lower pitch

    # D minor scale notes
    d_minor = ['D', 'E', 'F', 'G', 'A', 'Bb', 'C']

    # Map the scaled diameter percentage to a note
    note = mymidi.scale_to_note(scale_pct, d_minor)

    # Convert the note to a MIDI pitch value
    midi_pitch = mymidi.note_to_midi_pitch(note)

    return midi_pitch

# Function to convert diameter to velocity (larger diameter = louder sound)
def diameter_to_velocity(diameter):
    # Scale the diameter to a percentage where larger diameters map to higher velocities
    scale_pct = mymidi.linear_scale_pct(10, 96.7, diameter)  # Higher diameter = higher velocity

    # Scale the percentage to the MIDI velocity range (0 to 127)
    midi_velocity = int(scale_pct * 127)

    return midi_velocity

# Create a song list that will contain all the MIDI notes
note_list = []

# Iterate through each data point and create the notes
for d in my_data_timed:
    note_list.append([
        d['beat'] - start_time,                # time (relative to start_time)
        data_to_pitch_tuned(d['diameter']),    # pitch (larger crater = lower pitch)
        diameter_to_velocity(d['diameter']),   # velocity (larger crater = louder)
        2                                      # duration in beats
    ])

# Add the track with the notes to the MIDI file
mymidi.add_track(note_list)

# Output and save the MIDI file
mymidi.save_midi()


77 0.0 2 17
82 0.35 2 0
81 0.69 2 5
81 0.85 2 4
82 0.9600000000000001 2 2
81 1.04 2 7
81 1.11 2 6
82 1.1500000000000001 2 0
82 1.34 2 2
81 1.3800000000000001 2 7
76 1.4200000000000002 2 19
81 1.54 2 8
79 1.6500000000000001 2 12
82 1.77 2 0
82 1.92 2 2
81 2.0 2 6
72 2.19 2 31
69 2.92 2 39
82 2.95 2 3
82 3.0700000000000003 2 4
43 3.11 2 110
79 3.41 2 12
82 4.069999999999999 2 0
82 4.18 2 4
82 4.68 2 1
82 4.76 2 1
82 5.1 2 0
81 5.14 2 7
82 5.18 2 1
81 5.22 2 8
82 5.409999999999999 2 3
50 5.4799999999999995 2 89
81 5.9399999999999995 2 4
82 6.14 2 0
67 6.14 2 43
79 6.4799999999999995 2 10
81 6.56 2 8
76 6.67 2 19
82 6.79 2 0
82 6.79 2 3
82 7.09 2 2
81 7.17 2 8
82 7.25 2 1
65 7.359999999999999 2 48
79 7.859999999999999 2 13
82 8.209999999999999 2 0
81 8.32 2 9
82 8.4 2 0
70 8.44 2 35
82 8.51 2 1
82 8.969999999999999 2 1
74 9.59 2 26
82 9.7 2 2
82 9.78 2 4
77 10.28 2 15
81 10.969999999999999 2 5
55 11.0 2 79
79 11.16 2 10
82 11.389999999999999 2 1
81 11.85 2 7
81 12.04 2 4
82 13.0 2 3
82 13.